# UNET Introduction

- U-Net 모델은 binary segmentation하여 배경과 물체를 pixel-wise 분류하는 Simple한 Fully Convolutional Neural Network이다.
- 크게 두 부분으로 구성되어진다.
- 1) Contracting Path : Conv layer와 Downsampling (MaxPooling 또는 Cov2DTranspose)로 사이즈를 줄여가며 Feature를 뽑아낸다.
- 2) Expanding Path : Upsampling layer를 통해 사이즈를 복원한다.
- 그리고 2 부분이 같은 사이즈의 레벨에서 Contracting에서 저장된 feature를 Concatenation을 통해 Expanding에 붙여준다.
- 마지막에는 Conv layer를 통해 1x1 1channel을 통해 sigmoid로 분류를 한다.
</br>

![Unet.png](https://github.com/tenjumh/AI_Innovation_School/blob/master/UNET/image/Unet.png?raw=True)

## Imports module

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.losses import binary_crossentropy
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from keras import backend as K
from keras.utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle

## Dataset
- Ixford-IIIT Pet Datase을 사용할 것이다.
- 각 클래스당 200여개의 이미지로 개와 고양이 37 클래스가 포함되어 있다.
- 데이타셋은 세그멘테이션 마스크와 바운딩 박스 레이블이 포함되어 있다.
![dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/pet_annotations.jpg)

- 참고 URL : http://www.robots.ox.ac.uk/~vgg/data/pets/
- Download : Dataset and Groundtruth data

## Generators

In [2]:
def image_generator(files, batch_size=32, sz=(256,256)):
    while True:
        # 램덤으로 배치 추출
        batch = np.random.choice(files, size=batch_size)
        
        # input, output의 담을 리스트 준비
        batch_x = []
        batch_y = []
        
        for f in batch:
            # mask를 뽑기
            mask = Image.open(f'./data/annotations/trimaps/{f[:-4]}.png')
            mask = np.array(mask.resize(sz))          

The intersection over union (IoU) metric is a simple metric used to evaluate the performance of a segmentation algorithm. Given two masks $x_{true}, x_{pred}$ we evaluate 

$$IoU = \frac{y_{true} \cap y_{pred}}{y_{true} \cup y_{pred}}$$